# EDA > Combine

<div class="alert alert-info">Combine datasets using joins and set operations</div>

This notebook demonstrates how to combine datasets using various join operations and set operations. We use the `superheroes`, `publishers`, and `avengers` datasets to illustrate these concepts.

In [1]:
import polars as pl
import pyrsm as rsm

## setup pyrsm for autoreload
%reload_ext autoreload
%autoreload 2
%aimport pyrsm

# Load the Datasets

We'll use three small datasets to demonstrate combining data:
- **superheroes**: Contains superhero names, alignment, gender, and publisher
- **publishers**: Contains publisher names and founding years
- **avengers**: Contains Avengers team members (similar structure to superheroes)

In [2]:
superheroes = pl.read_parquet("https://github.com/radiant-ai-hub/pyrsm/raw/refs/heads/main/examples/data/data/superheroes.parquet")
superheroes

name,alignment,gender,publisher
str,str,str,str
"""Magneto""","""bad""","""male""","""Marvel"""
"""Storm""","""good""","""female""","""Marvel"""
"""Mystique""","""bad""","""female""","""Marvel"""
"""Batman""","""good""","""male""","""DC"""
"""Joker""","""bad""","""male""","""DC"""
"""Catwoman""","""bad""","""female""","""DC"""
"""Hellboy""","""good""","""male""","""Dark Horse Comics"""


In [3]:
rsm.md("https://raw.githubusercontent.com/radiant-ai-hub/pyrsm/refs/heads/main/examples/data/data/superheroes_description.md")

## Super heroes

### Variables

- name = super hero name
- alignment = good or bad character
- gender = male or female
- publisher = name of the comic publisher

<a href="http://stat545-ubc.github.io/bit001_dplyr-cheatsheet.html" target="_blank">Source</a>


In [4]:
publishers = pl.read_parquet("https://github.com/radiant-ai-hub/pyrsm/raw/refs/heads/main/examples/data/data/publishers.parquet")
publishers

publisher,yr_founded
str,i32
"""DC""",1934
"""Marvel""",1939
"""Image""",1992


In [5]:
rsm.md("https://raw.githubusercontent.com/radiant-ai-hub/pyrsm/refs/heads/main/examples/data/data/publishers_description.md")

## Comic publishers

### Variables

- publisher = name of the publisher
- yr_founded = year the publisher was founded

<a href="http://stat545-ubc.github.io/bit001_dplyr-cheatsheet.html" target="_blank">Source</a>


In [6]:
avengers = pl.read_parquet("https://github.com/radiant-ai-hub/pyrsm/raw/refs/heads/main/examples/data/data/avengers.parquet")
avengers

name,alignment,gender,publisher
str,str,str,str
"""Thor""","""good""","""male""","""Marvel"""
"""Iron Man""","""good""","""male""","""Marvel"""
"""Hulk""","""good""","""male""","""Marvel"""
"""Hawkeye""","""good""","""male""","""Marvel"""
"""Black Widow""","""good""","""female""","""Marvel"""
"""Captain America""","""good""","""male""","""Marvel"""
"""Magneto""","""bad""","""male""","""Marvel"""


In [7]:
rsm.md("https://raw.githubusercontent.com/radiant-ai-hub/pyrsm/refs/heads/main/examples/data/data/avengers_description.md")

## Avengers

### Variables

- name = super hero name
- alignment = good or bad character
- gender = male or female
- publisher = name of the comic publisher

Note: There is a deliberate mistake in the data. See the help file in the _Data > Combine_ tab

# Join Operations

Join operations combine two datasets based on matching values in one or more columns. The `publisher` column is the key that links superheroes to publishers.

Note that:
- **Hellboy** is published by "Dark Horse Comics", which is not in the publishers table
- **Image** is a publisher with no superheroes in our superheroes table

## Inner Join

Returns all rows from the left table (`superheroes`) that have matching values in the right table (`publishers`), along with all columns from both tables.

Only rows with matches in **both** datasets are retained. Notice that Hellboy is excluded because "Dark Horse Comics" is not in the publishers table.

In [8]:
superheroes.join(publishers, on="publisher", how="inner")

name,alignment,gender,publisher,yr_founded
str,str,str,str,i32
"""Magneto""","""bad""","""male""","""Marvel""",1939
"""Storm""","""good""","""female""","""Marvel""",1939
"""Mystique""","""bad""","""female""","""Marvel""",1939
"""Batman""","""good""","""male""","""DC""",1934
"""Joker""","""bad""","""male""","""DC""",1934
"""Catwoman""","""bad""","""female""","""DC""",1934


## Left Join

Returns all rows from the left table (`superheroes`), and all columns from both tables. Rows in the left table without a match in the right table will have `null` values for the right table's columns.

All superheroes are kept. Hellboy remains in the result but has `null` for `yr_founded` because his publisher (Dark Horse Comics) is not in the publishers table.

In [9]:
superheroes.join(publishers, on="publisher", how="left")

name,alignment,gender,publisher,yr_founded
str,str,str,str,i32
"""Magneto""","""bad""","""male""","""Marvel""",1939
"""Storm""","""good""","""female""","""Marvel""",1939
"""Mystique""","""bad""","""female""","""Marvel""",1939
"""Batman""","""good""","""male""","""DC""",1934
"""Joker""","""bad""","""male""","""DC""",1934
"""Catwoman""","""bad""","""female""","""DC""",1934
"""Hellboy""","""good""","""male""","""Dark Horse Comics""",null


## Right Join

Returns all rows from the right table (`publishers`), and all columns from both tables. Rows in the right table without a match in the left table will have `null` values for the left table's columns.

All publishers are preserved. The Image publisher appears even though no superhero in our data represents it (with `null` values for superhero columns).

In Polars, we achieve this by swapping the tables and using a left join, then reordering columns:

In [10]:
# Right join: keep all publishers
publishers.join(superheroes, on="publisher", how="left").select(
    ["name", "alignment", "gender", "publisher", "yr_founded"]
)

name,alignment,gender,publisher,yr_founded
str,str,str,str,i32
"""Batman""","""good""","""male""","""DC""",1934
"""Joker""","""bad""","""male""","""DC""",1934
"""Catwoman""","""bad""","""female""","""DC""",1934
"""Magneto""","""bad""","""male""","""Marvel""",1939
"""Storm""","""good""","""female""","""Marvel""",1939
"""Mystique""","""bad""","""female""","""Marvel""",1939
null,null,null,"""Image""",1992


## Full (Outer) Join

Returns all rows from both tables, keeping rows and columns that appear in either. Unmatched rows from either table will have `null` values for the other table's columns.

Both Hellboy (no matching publisher) and Image (no matching superhero) are retained with `null` values where matches don't exist.

In [11]:
superheroes.join(publishers, on="publisher", how="full", coalesce=True)

name,alignment,gender,publisher,yr_founded
str,str,str,str,i32
"""Magneto""","""bad""","""male""","""Marvel""",1939
"""Storm""","""good""","""female""","""Marvel""",1939
"""Mystique""","""bad""","""female""","""Marvel""",1939
"""Batman""","""good""","""male""","""DC""",1934
"""Joker""","""bad""","""male""","""DC""",1934
"""Catwoman""","""bad""","""female""","""DC""",1934
"""Hellboy""","""good""","""male""","""Dark Horse Comics""",null
null,null,null,"""Image""",1992


## Semi Join

Returns all rows from the left table that have a match in the right table, but **only keeps columns from the left table**. This is useful for filtering one table based on the presence of matching keys in another.

Returns superheroes that have a matching publisher, but no publisher data columns are included. Hellboy is excluded.

In [12]:
superheroes.join(publishers, on="publisher", how="semi")

name,alignment,gender,publisher
str,str,str,str
"""Magneto""","""bad""","""male""","""Marvel"""
"""Storm""","""good""","""female""","""Marvel"""
"""Mystique""","""bad""","""female""","""Marvel"""
"""Batman""","""good""","""male""","""DC"""
"""Joker""","""bad""","""male""","""DC"""
"""Catwoman""","""bad""","""female""","""DC"""


## Anti Join

Returns all rows from the left table that do **not** have a match in the right table. Only columns from the left table are kept.

Returns only unmatched records from the superheroes table. In this case, only Hellboy is returned because his publisher (Dark Horse Comics) is not in the publishers table.

In [13]:
superheroes.join(publishers, on="publisher", how="anti")

name,alignment,gender,publisher
str,str,str,str
"""Hellboy""","""good""","""male""","""Dark Horse Comics"""


# Set Operations

For datasets with identical (or similar) structure, we can use set operations to combine them. We'll use `superheroes` and `avengers` which have the same columns.

## Vertical Concatenation (Bind Rows)

Stacks datasets vertically, combining all rows from both tables. This is equivalent to `rbind` in R or `UNION ALL` in SQL.

Note: Magneto appears in both datasets, so he appears twice in the result.

In [14]:
pl.concat([superheroes, avengers], how="vertical")

name,alignment,gender,publisher
str,str,str,str
"""Magneto""","""bad""","""male""","""Marvel"""
"""Storm""","""good""","""female""","""Marvel"""
"""Mystique""","""bad""","""female""","""Marvel"""
"""Batman""","""good""","""male""","""DC"""
"""Joker""","""bad""","""male""","""DC"""
…,…,…,…
"""Hulk""","""good""","""male""","""Marvel"""
"""Hawkeye""","""good""","""male""","""Marvel"""
"""Black Widow""","""good""","""female""","""Marvel"""


## Horizontal Concatenation (Bind Columns)

Places datasets side-by-side, combining columns. The datasets must have the same number of rows and column names must be unique.

Here we combine the first 3 superheroes (excluding the publisher column) with the publishers table:

In [15]:
pl.concat([superheroes.head(3).select(["name", "alignment", "gender"]), publishers], how="horizontal")

name,alignment,gender,publisher,yr_founded
str,str,str,str,i32
"""Magneto""","""bad""","""male""","""DC""",1934
"""Storm""","""good""","""female""","""Marvel""",1939
"""Mystique""","""bad""","""female""","""Image""",1992


## Union (Distinct Rows)

Combines datasets while removing duplicate rows. This is equivalent to `UNION` in SQL.

Magneto appears in both datasets but only once in the result.

In [16]:
pl.concat([superheroes, avengers], how="vertical").unique()

name,alignment,gender,publisher
str,str,str,str
"""Mystique""","""bad""","""female""","""Marvel"""
"""Black Widow""","""good""","""female""","""Marvel"""
"""Catwoman""","""bad""","""female""","""DC"""
"""Thor""","""good""","""male""","""Marvel"""
"""Captain America""","""good""","""male""","""Marvel"""
…,…,…,…
"""Hawkeye""","""good""","""male""","""Marvel"""
"""Batman""","""good""","""male""","""DC"""
"""Joker""","""bad""","""male""","""DC"""


## Intersect

Returns rows that appear in **both** datasets. This identifies duplicate/overlapping records.

Only Magneto appears in both superheroes and avengers.

In [17]:
# Rows that appear in both datasets
superheroes.join(avengers, on=superheroes.columns, how="semi")

name,alignment,gender,publisher
str,str,str,str
"""Magneto""","""bad""","""male""","""Marvel"""


## Set Difference

Returns rows from the first dataset that are **not** in the second dataset.

All superheroes except Magneto (who also appears in avengers).

In [18]:
# Rows in superheroes but not in avengers
superheroes.join(avengers, on=superheroes.columns, how="anti")

name,alignment,gender,publisher
str,str,str,str
"""Storm""","""good""","""female""","""Marvel"""
"""Mystique""","""bad""","""female""","""Marvel"""
"""Batman""","""good""","""male""","""DC"""
"""Joker""","""bad""","""male""","""DC"""
"""Catwoman""","""bad""","""female""","""DC"""
"""Hellboy""","""good""","""male""","""Dark Horse Comics"""


# Summary of Join Types

| Join Type | Keeps from Left | Keeps from Right | Use Case |
|-----------|-----------------|------------------|----------|
| Inner | Matched rows only | Matched rows only | Get only complete matches |
| Left | All rows | Matched rows only | Keep all from primary table |
| Right | Matched rows only | All rows | Keep all from secondary table |
| Full | All rows | All rows | Keep everything from both |
| Semi | Matched rows only | None (filter only) | Filter left by right |
| Anti | Unmatched rows only | None (filter only) | Find records without matches |

© Vincent Nijs (2026)